In [3]:
import torch
from torch import nn
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('data_sale_moeinmal.csv', header=None)
columns = ['date', 'id_prd_to_plc', 'id_br','amount', 'price']
df.columns = columns
df.head()


df['total_price'] = df['price'] * df['amount']


def make_year(column):
    year = column.split('-')[0]
    return int(year)

def make_month(column):
    month = column.split('-')[1]
    return int(month)


def make_day(column):
    day = column.split('-')[2][0:2]
    return int(day)

def make_date(column):
    date = column.split(" ")[0]
    return date

df['year'] = df['date'].apply(make_year)
df['month'] = df['date'].apply(make_month)
df['day'] = df['date'].apply(make_day)
df['date'] = df['date'].apply(make_date)

def convert_prd(column):
  if column == 401:
    return 1397
  if column == 901:
    return 1398
  if column == 1001:
    return 1399
  if column == 1101:
    return 1400
  if column == 1201:
    return 1401
  elif column == 1301:
    return 1402


df['id_prd_to_plc'] = df['id_prd_to_plc'].apply(convert_prd)
df.head(-5)

,date,id_prd_to_plc,id_br,amount,price,total_price,year,month,day
0,2018-11-18,1397,51138,1.0,1930000.0,1930000.0,2018,11,18
1,2018-11-18,1397,51138,1.0,1230000.0,1230000.0,2018,11,18
2,2018-11-18,1397,51138,1.0,1930000.0,1930000.0,2018,11,18
3,2018-11-18,1397,51138,1.0,1250000.0,1250000.0,2018,11,18
4,2018-11-18,1397,51138,1.0,2100000.0,2100000.0,2018,11,18
...,...,...,...,...,...,...,...,...,...
26039,2023-08-27,1402,51138,1.0,9400000.0,9400000.0,2023,8,27
26040,2023-08-27,1402,51138,1.0,6400000.0,6400000.0,2023,8,27
26041,2023-08-27,1402,51138,1.0,11550000.0,11550000.0,2023,8,27
26042,2023-08-28,1402,51138,1.0,5950000.0,5950000.0,2023,8,28


In [29]:
# Create date column for splitting
df['date'] = df['date'] +'-'+ df['id_prd_to_plc'].astype(str)

#Group by day
new_data = df.groupby(df['date'], as_index=False).sum(numeric_only=True)
new_data['id_br'] = 51238
# new_data = new_data.drop('id_inv', axis=1)
new_data.head()
def make_prd(column):
  id_prd = column.split('-')[3]
  return int(id_prd)

# Create again columns after make new df for each day sales
new_data['id_br'] = new_data['id_br'].astype(int)
new_data = new_data.drop(['year', 'month', 'day'], axis=1)
new_data['year'] = new_data['date'].apply(make_year)
new_data['month'] = new_data['date'].apply(make_month)
new_data['day'] = new_data['date'].apply(make_day)
new_data['id_prd_to_plc'] = new_data['date'].apply(make_prd)

new_data.head()

,date,id_prd_to_plc,id_br,amount,price,total_price,year,month,day
0,2018-11-18-1397-1397,1397,51238,24.0,49490000.0,49490000.0,2018,11,18
1,2018-11-19-1397-1397,1397,51238,1.0,2100000.0,2100000.0,2018,11,19
2,2018-11-20-1397-1397,1397,51238,3.0,9670000.0,9670000.0,2018,11,20
3,2018-11-21-1397-1397,1397,51238,6.0,20930000.0,20930000.0,2018,11,21
4,2018-11-22-1397-1397,1397,51238,12.0,38750000.0,38750000.0,2018,11,22


In [41]:
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

scaler = MinMaxScaler()
X = new_data[['id_prd_to_plc', 'year', 'month', 'day']].values
y = new_data['total_price'].values

X = scaler.fit_transform(X)
# X = torch.tensor(X, dtype=torch.float32)
# X = X.view(X.shape[0], 1, X.shape[1])
# y = torch.tensor(y, dtype=torch.float32)

# dataset = TensorDataset(X, y)
# dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [82]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Adjust test_size as needed
X_train = X_train.reshape(X_train.shape[0], 1, -1)
X_test = X_test.reshape(X_test.shape[0], 1, -1)

# Create data loaders for the training and test sets
batch_size = 1  # You can adjust this batch size
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32))
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) 

## Create model

In [83]:
import torch.nn as nn

class SalesPredictionModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(SalesPredictionModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)  # Output size is 1 for regression

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Get the last time step's output
        return out

In [107]:
model = SalesPredictionModel(4, 68, 6)

In [108]:
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_dataloader:
        targets = targets.view(-1, 1)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        for inputs, targets in test_dataloader:
            targets = targets.view(-1, 1)
            y_preds = model(inputs)
            loss_test = criterion(y_preds, targets)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Test_loss: {loss_test.item():.4f}')

Epoch [1/20], Loss: 265369984.0000, Test_loss: 72629992.0000
Epoch [2/20], Loss: 93959984.0000, Test_loss: 72629984.0000
Epoch [3/20], Loss: 92649976.0000, Test_loss: 72629976.0000
Epoch [4/20], Loss: 12009968.0000, Test_loss: 72629968.0000
Epoch [5/20], Loss: 34724960.0000, Test_loss: 72629960.0000
Epoch [6/20], Loss: 15254955.0000, Test_loss: 72629952.0000
Epoch [7/20], Loss: 61819948.0000, Test_loss: 72629952.0000
Epoch [8/20], Loss: 178299936.0000, Test_loss: 72629944.0000


In [87]:
import numpy as np
np.sqrt(4425784215207936)

66526567.739572555

In [91]:
month = new_data.iloc[1056: 1087]
month.head()

,date,id_prd_to_plc,id_br,amount,price,total_price,year,month,day
1056,2022-04-04-1401-1401,1401,51238,9.0,50950000.0,50950000.0,2022,4,4
1057,2022-04-05-1401-1401,1401,51238,4.0,28800000.0,28800000.0,2022,4,5
1058,2022-04-06-1401-1401,1401,51238,7.0,42250000.0,42250000.0,2022,4,6
1059,2022-04-07-1401-1401,1401,51238,12.0,70100000.0,70100000.0,2022,4,7
1060,2022-04-08-1401-1401,1401,51238,16.0,82500000.0,82500000.0,2022,4,8


In [100]:
X_month = month[['id_prd_to_plc', 'year', 'month', 'day']].values
y_month = month['total_price'].values
x_month = torch.tensor(scaler.transform(X_month), dtype=torch.float32).reshape(x_month.shape[0], 1, -1)

y_predictions = model(x_month)